In [1]:
!del /Q utils\__pycache__

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics

In [4]:
from sklearn.metrics import mean_squared_error

# Data

In [5]:
ds = GhlKasperskyDataset()
ds.shake_not_stir()

In [6]:
train, _, _ = next(ds.train_generator())
valid, _, _ = next(ds.valid_generator())

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
for data, _, _ in ds.train_generator():
    scaler.partial_fit(data)

In [9]:
SEED = 7145

# LSTM

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Conv1D, ConvLSTM1D
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import set_random_seed

In [11]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [12]:
n_features = train.shape[1]
valid_size = 0.2
res_lstm = pd.DataFrame(dtype='float')

In [13]:
checkpoint = ModelCheckpoint('best_ghl_lstm.hdf5',
                             monitor='val_mse', 
                             verbose=0, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_mse',
                          patience=2,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=0,
                         )

callbacks_list = [checkpoint, earlystop]

In [14]:
for n_steps in tqdm([2, 4, 8, 16, 32]):
    for rnn_dim in tqdm([32, 64, 128, 256]):
        X, y = split_sequences(scaler.transform(train), n_steps)
        
        train_n = round(X.shape[0] * (1 - valid_size))
        X_train = X[:train_n]
        y_train= y[:train_n]
        X_valid = X[train_n:]
        y_valid = y[train_n:]

        clear_session()
        set_random_seed(SEED)
        model = Sequential()
        model.add(LSTM(rnn_dim, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(rnn_dim, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
        model.fit(X_train, y_train,
                  epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=callbacks_list,
                  verbose=0,
                 )
        
        y_ = model.predict(X_valid)
        res_lstm.loc[n_steps, rnn_dim] = mean_squared_error(y_valid, y_)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
res_lstm

,32,64,128
2,0.116756,0.101350,0.121519
4,0.133456,0.106358,0.118350
8,0.117916,0.120674,0.126802
16,0.136642,0.134246,0.110592
32,0.166581,0.135608,0.130117


In [16]:
res_lstm.mean(axis=0)

32     0.134270
64     0.119647
128    0.121476
dtype: float64

In [17]:
res_lstm.mean(axis=1)

2     0.113209
4     0.119388
8     0.121797
16    0.127160
32    0.144102
dtype: float64

# GRU

In [18]:
res_gru = pd.DataFrame(dtype='float')

In [19]:
checkpoint = ModelCheckpoint('best_ghl_gru.hdf5',
                             monitor='val_mse', 
                             verbose=0, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_mse',
                          patience=2,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=0,
                         )

callbacks_list = [checkpoint, earlystop]

In [20]:
for n_steps in tqdm([2, 4, 8, 16, 32]):
    for rnn_dim in tqdm([32, 64, 128, 256]):
        X, y = split_sequences(scaler.transform(train), n_steps)
        
        train_n = round(X.shape[0] * (1 - valid_size))
        X_train = X[:train_n]
        y_train= y[:train_n]
        X_valid = X[train_n:]
        y_valid = y[train_n:]

        clear_session()
        set_random_seed(SEED)
        model = Sequential()
        model.add(GRU(rnn_dim, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(GRU(rnn_dim, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
        model.fit(X_train, y_train,
                  epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=callbacks_list,
                  verbose=0,
                 )
        
        y_ = model.predict(X_valid)
        res_gru.loc[n_steps, rnn_dim] = mean_squared_error(y_valid, y_)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
res_gru

,32,64,128
2,0.115627,0.096118,0.117307
4,0.137974,0.126135,0.116888
8,0.138558,0.130839,0.128089
16,0.148013,0.103846,0.106176
32,0.124888,0.099594,0.099551


In [22]:
res_gru.mean(axis=0)

32     0.133012
64     0.111306
128    0.113602
dtype: float64

In [23]:
res_gru.mean(axis=1)

2     0.109684
4     0.126999
8     0.132495
16    0.119345
32    0.108011
dtype: float64

In [26]:
res_gru.values[:-3]

array([[0.1156268 , 0.09611776, 0.11730716],
       [0.1379741 , 0.12613483, 0.11688779]])